<a href="https://colab.research.google.com/github/tozanni/Data_Science_Notebooks/blob/main/Decenso_por_gradiente.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Descenso por gradiente paso a paso

Este cuaderno ha sido traducido y adaptado de la siguiente referencia.

[Alan Reiner - Introduction to Custom Gradient Descent in Tensorflow 2.0](https://github.com/etotheipi/toptal_tensorflow_blog_post/blob/dev/simple_height_vs_weight/tf_grad_desc_intro.ipynb)

In [ ]:
import os
import shutil
import sys
import seaborn as sns
import tensorflow as tf
import sklearn
from tensorflow import keras
import pandas as pd
import numpy as np

from IPython.display import display, Image
import matplotlib.pyplot as plt
import matplotlib.animation as animation
%matplotlib inline

Leer el archivo

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/tozanni/Data_Science_Notebooks/main/height_v_weight.csv')
df = df.drop(labels=' Gender', axis=1)
df.head(5)

In [ ]:
def calc_mean_sq_error(df, slope, intercept):
    """
    Calcula el error cuadrático medio (MSE) de la línea como predictor de pesos
    """
    hs, ws = df['Height'].values, df['Weight'].values
    diffs = (slope * hs + intercept) - ws
    mse = np.mean(diffs**2) 
    return mse
    
    
def draw_scatter(df, slope=None, intercept=None, ax=None):
    if ax is None:
        _,ax = plt.subplots(figsize=(4,4))
        
    sns.scatterplot(df['Height'], df['Weight'], ax=ax, label='Sample Points')
    ax.set_xlabel('Height (inches)')
    ax.set_ylabel('Weight (lbs)')
    ax.set_title('Male Height vs. Weight Sample')
    
    # For this exercise we're going to hardcode various parameters for simplicity
    h0,h1 = 60, 78
    w0,w1 = 130, 260
    ax.set_xlim(h0, h1)
    ax.set_ylim(w0, w1)
    ax.set_aspect((h1-h0)/(w1-w0))
    
    if slope is not None and intercept is not None:
        pred_w0, pred_w1 = (slope*h0 + intercept, slope*h1 + intercept)
        ax.plot([h0, h1], [pred_w0, pred_w1], 'r-.', label='Fitted Line')
        mse = calc_mean_sq_error(df, slope, intercept)
        eqn_str = f'w = {slope:.2f} * h + {intercept:.1f}'
        ax.set_title(f'Loss: MSE={mse:.1f}\n{eqn_str}')
        
    ax.legend(loc='upper left')
    return ax

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(10,5))
draw_scatter(df, slope=4,  intercept = -120, ax=axs[0])
draw_scatter(df, slope=2,  intercept =   70, ax=axs[1])
draw_scatter(df, slope=3,  intercept =  -30, ax=axs[2])
plt.tight_layout(2)

In [ ]:
# Encontremos la solución analítica usando sklearn
from sklearn.linear_model import LinearRegression
lin_reg = LinearRegression()
lin_reg.fit(df['Height'].values.reshape([-1, 1]), df['Weight'].values)

# Veamos los parámetros óptimos del modelo y su pérdida MSE
true_slope, true_intercept = lin_reg.coef_[0], lin_reg.intercept_
min_mse = calc_mean_sq_error(df, true_slope, true_intercept)

_ = draw_scatter(df, slope=true_slope, intercept=true_intercept)

Usando Tensorflow y Gradient Descent

Para un problema tan simple con una solución analítica, no necesitamos usar el descenso de gradiente. Pero el objetivo aquí es presentar la función de diferenciación automática de Tensorflow en un problema simple para tener una idea de su mecánica sin complejidad. También veremos que se trata de soluciones razonables, incluso si en realidad no son óptimas.

In [ ]:
def tf_calc_mse_verbose(heights, weights, slope, intercept):
    """
    Este método es básicamente idéntico a calc_mse() de antes, pero
     necesita usar solo operaciones definidas en tensores y métodos tf.*.
    """
    predictions = tf.add(tf.multiply(slope, heights), intercept)
    errors = tf.subtract(predictions, weights)
    mse = tf.reduce_mean(tf.square(errors))
    return mse
    
def tf_calc_mse_clean(heights, weights, slope, intercept):
    """
    Tensor algebra can be implemented with standard operators, simplifying
    the expressions.  This method is identical to the one above.
    """
    predictions = slope * heights + intercept
    errors = predictions - weights
    mse = tf.reduce_mean(errors**2)
    return mse

In [ ]:
def draw_linreg_progress(df, slope, intercept, mse_hist, min_mse, file_out=None, n_iter=25, fig=None, axs=None):
    if fig is None or axs is None:
        fig, axs = plt.subplots(1, 2, figsize=(8,4))
        
    draw_scatter(df, slope, intercept, ax=axs[0])
    
    # Draw loss-chart.  Hardcode a few more parameters for simplicity
    hmin, hmax = 0, n_iter
    wmin, wmax = 0, 2000
    axs[1].plot(range(len(mse_hist)), mse_hist, 'b-', marker='o', label='Computed Loss')
    axs[1].plot([hmin, hmax], [min_mse, min_mse], 'g-.', label='Minimum Possible Loss')
    axs[1].set_xlim(hmin, hmax)
    axs[1].set_ylim(wmin, wmax)
    axs[1].set_xlabel('Iteration')
    axs[1].set_ylabel('Loss (MSE)')
    axs[1].legend(loc='upper right')
    axs[1].set_aspect(hmax/float(wmax))
    
    # Update the axis titles
    mse = calc_mean_sq_error(df, slope, intercept) if len(mse_hist)==0 else mse_hist[-1]
    axs[0].set_title('Male Heights & Weights')
    eqn_str = f'w = {slope:.2f} * h + {intercept:.1f}'
    axs[1].set_title(f'Loss: MSE={mse:.1f}\n{eqn_str}')
    #plt.tight_layout(3.0)
    
    if file_out:
        fig.savefig(file_out)
        
    return fig, axs
    
_ = draw_linreg_progress(df, true_slope, true_intercept, [], min_mse)



In [ ]:
def run_gradient_descent(df, init_slope, init_icept, n_iter=25, learning_rate=2e-5, dir_name=None):
    """
     Proporciona una estimación inicial de la pendiente y la intersección, el descenso de la pendiente se ajustará
     y proporcionará una solución cercana a la óptima
    """
    
    Hs, Ws = df['Height'].values, df['Weight'].values
    
    # Las variables que seran parte de los cálculos de gradiente deben de convertirse
    # mediante la funcion tf.Variable de TensorFlow

    tf_slope = tf.Variable(init_slope, dtype='float32') 
    tf_icept = tf.Variable(init_icept, dtype='float32') 
    
    # Acumular el historial de pérdida de cada época
    loss_hist = []
    shutil.rmtree(dir_name, ignore_errors=True)
    
    fig, axs = plt.subplots(1, 2, figsize=(9,4))
    
    for i in range(n_iter):
        
        # tf.GradientTape() es el objeto que lleva registro de todos los cálculos 
        # de tensores diferenciables en el bloque de código

        with tf.GradientTape() as tape:
            # Indicar a gradientTape que queremos llevar registro de la pendiente
            # y del intercepto

            tape.watch((tf_slope, tf_icept))

            # Calcular la pérdida 
            predictions = tf_slope * Hs + tf_icept
            errors = predictions - Ws
            loss = tf.reduce_mean(errors**2)

        #########################################################################
        # Magia!  Obtener la derivada de la pérdida con respecto a los parámetros
        dloss_dparams = tape.gradient(loss, [tf_slope, tf_icept])
        #########################################################################

        # Dado que no normalizamos los datos y la pendiente tiene diferente magnitud
        # que el intercepto, debemos ajustar los gradientes del intercepto por una 
        # constante razonable (en este caso 1000)
        tf_slope = tf_slope - learning_rate * dloss_dparams[0]
        tf_icept = tf_icept - learning_rate * dloss_dparams[1] * 1000.0
            
        # Registrar y graficar el valor de la pérdida
        loss_hist.append(loss)
        if dir_name:
            os.makedirs(dir_name, exist_ok=True)
            fout = os.path.join(dir_name, f'img_{i:03d}.png')
            axs[0].clear()
            axs[1].clear()
            draw_linreg_progress(df, tf_slope, tf_icept, loss_hist, min_mse, n_iter=n_iter, file_out=fout, fig=fig, axs=axs)
        
    return tf_slope.numpy(), tf_icept.numpy(), loss_hist[-1]
        


¡Veamos qué tan bien funciona!

Resulta que la superficie de pérdida es un dificil de lo que cabría esperar, especialmente porque no normalizamos nuestros datos de antemano. Si bien esto debería funcionar teóricamente, terminamos necesitando algunas tasas de aprendizaje por parámetro súper dinámicas para que funcione a partir de valores iniciales elegidos arbitrariamente.

Para evitar complicar esto en exceso, simplemente lo ejecutaremos a partir de una suposición inicial razonable de los parámetros y dejaremos que ajuste la respuesta por nosotros. En la mayoría de los problemas de descenso de gradiente.

In [ ]:
slope1, int1, loss1 = run_gradient_descent(df, true_slope, -180, dir_name='imgs_descent_1')